In [ ]:
'''Общая концепция машинного обучения.'''
# Предобработка признаков
## Для текстовых признаков
stop_words = list(stopwords.words('russian')) 
text = [...] # список признаков
## Числовые признаки
numerical = [...] # список признаков
num_pipe = Pipeline([('scaler', StandardScaler())])
## Категориальные признаки
categorical = [...] # список признаков
cat_pipe = Pipeline([('ohe', OneHotEncoder())])
## Итоговый пайплайн преобработки
preprocessor = ColumnTransformer(
    transformers = [(x, TfidfVectorizer(stop_words=stop_words), x) for x in text] + \
                   [('num', num_pipe, numerical), ('cat', cat_pipe, categorical)], 
    remainder='passthrough')

'''В коде выше мы используем оператор + для соединения двух частей списка 
[(x, TfidfVectorizer(stop_words=stop_words), x) for x in text] и 
[('num', num_pipe, numerical), ('cat', cat_pipe, categorical)] в один общий список, 
который затем используется в качестве аргумента для параметра transformers в ColumnTransformer.'''

# GridSearchCV подбор параметров
clf = Pipeline([('preprocessor', preprocessor),
                ('svm', SVMSMOTE(random_state=42)),
                ('model', CatBoostClassifier(class_weights=dict(zip(np.unique(target_train), 
                                                                    compute_class_weight(class_weight='balanced',
                                                                                         classes=np.unique(target_train), 
                                                                                         y=target_train))), 
                                             silent=True, random_seed=42))])
params = {'model__depth':[5,8,10], 'model__iterations': [100, 150, 200]} 
gs = GridSearchCV(clf, params, scoring='f1', n_jobs=-1, cv=skf.split(features_train, target_train))
print(gd.best_params_)

# Обучение модели
model = Pipeline([('sampling', SVMSMOTE(random_state=42)),
                  ('model', CatBoostClassifier(depth=10, iterations=200))])

features_train = preprocessor.transform(features_train) # Предобработка признаков
model.fit(features_train, target_train) # Обучение модели

# сохранение модели
dump(model, 'model.joblib')
